In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [ ]:
import pandas as pd
from langchain_core.tools import tool
from langchain_core.runnables import Runnable
from langchain.chat_models import init_chat_model

# Sample DataFrame
df = pd.DataFrame({
    "Product": ["Apple", "Banana", "Orange"],
    "Price": [1.2, 0.8, 1.5],
    "Quantity": [100, 150, 120]
})
from langchain_core.prompts import ChatPromptTemplate

# System prompt to restrict LLM behavior
prompt = ChatPromptTemplate.from_template("""
You are a Python data analyst.
You are ONLY allowed to call the function: query_dataframe(code: str)
Use one line of pandas code and store the output in a variable named `result`.

Example:
result = df.loc[df['Price'].idxmax()]

The dataframe is named `df` and has these columns: Product, Price, Quantity.

Do NOT invent any function name. Only call `query_dataframe`.

User question: {question}
""")

# Format the input for the LLM
input_msg = prompt.format_messages(question="Which product has the highest price?")

# Now invoke LLM with the proper context
response = model.invoke(input_msg)

# Tool definition
@tool
def query_dataframe(code: str) -> str:
    """Execute a line of pandas code using 'df'. Store the result in a variable called 'result'."""
    local_vars = {'df': df}
    try:
        exec(code, {}, local_vars)
        return str(local_vars.get("result", "No result found"))
    except Exception as e:
        return f"Error: {e}"

# LLM setup with tool binding
model = init_chat_model("llama3-8b-8192", model_provider="groq").bind_tools([query_dataframe])
chain: Runnable = model

def run_agent(question: str):
    print(f"\n💬 Question: {question}")

    input_msg = prompt.format_messages(question=question)
    response = model.invoke(input_msg)

    if hasattr(response, 'tool_calls') and response.tool_calls:
        tool_call = response.tool_calls[0]
        #print(f"\n🔧 Tool Called: {tool_call.name}")
        print(f"🧠 Generated Code:\n{tool_call["args"]['code']}")

        result = query_dataframe(tool_call["args"]['code'])
        print(f"\n📊 Result:\n{result}")

        # Final explanation
        follow_up = f"The user asked: '{question}'. The result is: {result}"
        explanation = model.invoke(follow_up)
        print(f"\n🗣️ Final Answer:\n{explanation.content}")
    else:
        print("\n❌ No tool call made.")
        print("LLM said:\n", response.content)


# 🚀 Run
run_agent("Which product has the highest price?")



💬 Question: Which product has the highest price?
🧠 Generated Code:
df.loc[df['Price'].idxmax()]

📊 Result:
No result found


C:\Users\nethi\AppData\Local\Temp\ipykernel_27340\3357062014.py:39: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = query_dataframe(tool_call["args"]['code'])



🗣️ Final Answer:



In [32]:
result = df.loc[df['Price'].idxmax()]


In [33]:
result

Product     Orange
Price          1.5
Quantity       120
Name: 2, dtype: object

In [26]:
response.tool_calls[0]["args"]["code"]# The generated code

"result = df.loc[df['Price'].idxmax()]"

In [60]:
import pandas as pd
from langchain_core.tools import tool
from langchain_core.runnables import Runnable
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate

# Sample DataFrame
df = pd.read_csv("D:\K_AgentiAI\DASH-AI_Agent\Data\CSV\supermarket_sales.csv")
cols=df.columns.to_list()
# Define the tool
@tool
def query_dataframe(code: str) -> str:
    """Takes a pandas query (e.g., df['Price'].max()) and executes it on the DataFrame 'df'.
    You do NOT need to include `result =` — that will be handled inside this tool."""
    local_vars = {'df': df}
    print(local_vars)
    try:
        exec(f"result = {code}", {}, local_vars)
        return str(local_vars.get("result", "No result found"))
    except Exception as e:
        return f"Error: {e}"

# Initialize LLM with the tool
model = init_chat_model("llama3-8b-8192", model_provider="groq").bind_tools([query_dataframe])
chain: Runnable = model

# Prompt instructing model to return ONLY the pandas query string
prompt = ChatPromptTemplate.from_template("""
You are a pandas expert. Given a question about a DataFrame with columns: {cols}.
                      
User question: {question}
""")

# Main function to run agent
def run_agent(question: str):
    print(f"\n💬 Question: {question}")

    # Generate pandas code using LLM
    input_msg = prompt.format_messages(question=question,cols=cols)
    response = model.invoke(input_msg)

    if hasattr(response, 'tool_calls') and response.tool_calls:
        tool_call = response.tool_calls[0]
        pandas_query = tool_call["args"]['code']
        print(f"\n🧠 Generated pandas code:\n{pandas_query}")

        # Run the tool
        result = query_dataframe.invoke({"code": pandas_query})

        print(f"\n📊 Query Result:\n{result}")

        # Second LLM call to explain result
        explanation_prompt = f"The user asked: '{question}'. The result of executing '{pandas_query}' is: {result},Please now give the final response to the user.Dont use any tool call now"
        print(f"\n🔍 Explanation Prompt:\n{explanation_prompt}")
        explanation = model.invoke(explanation_prompt)
        # Print the final explanation
        print(f"\n🗣️ Final Answer:\n{explanation.content}")
    else:
        print("\n❌ No tool call made.")
        print("LLM response:\n", response.content)

run_agent("what is total count of invoice id generated from branch A ?")

<>:8: SyntaxWarning: invalid escape sequence '\K'
<>:8: SyntaxWarning: invalid escape sequence '\K'
C:\Users\nethi\AppData\Local\Temp\ipykernel_27340\3379807604.py:8: SyntaxWarning: invalid escape sequence '\K'
  df = pd.read_csv("D:\K_AgentiAI\DASH-AI_Agent\Data\CSV\supermarket_sales.csv")



💬 Question: what is total count of invoice id generated from branch A ?

🧠 Generated pandas code:
df[df['Branch'] == 'A']['Invoice ID'].count()
{'df':       Invoice ID Branch       City Customer type  Gender  \
0    750-67-8428      A     Yangon        Member  Female   
1    226-31-3081      C  Naypyitaw        Normal  Female   
2    631-41-3108      A     Yangon        Normal    Male   
3    123-19-1176      A     Yangon        Member    Male   
4    373-73-7910      A     Yangon        Normal    Male   
..           ...    ...        ...           ...     ...   
995  233-67-5758      C  Naypyitaw        Normal    Male   
996  303-96-2227      B   Mandalay        Normal  Female   
997  727-02-1313      A     Yangon        Member    Male   
998  347-56-2442      A     Yangon        Normal    Male   
999  849-09-3807      A     Yangon        Member  Female   

               Product line  Unit price  Quantity   Tax 5%      Total  \
0         Health and beauty       74.69         7  26.

In [69]:
run_agent(" Get the Most Common Product Line Purchased by Customers in Branch 'A' ")


💬 Question:  Get the Most Common Product Line Purchased by Customers in Branch 'A' 

🧠 Generated pandas code:
df.loc[df['Branch'] == 'A'].groupby('Product line')['Invoice ID'].count().sort_values(ascending=False).head(1)
{'df':       Invoice ID Branch       City Customer type  Gender  \
0    750-67-8428      A     Yangon        Member  Female   
1    226-31-3081      C  Naypyitaw        Normal  Female   
2    631-41-3108      A     Yangon        Normal    Male   
3    123-19-1176      A     Yangon        Member    Male   
4    373-73-7910      A     Yangon        Normal    Male   
..           ...    ...        ...           ...     ...   
995  233-67-5758      C  Naypyitaw        Normal    Male   
996  303-96-2227      B   Mandalay        Normal  Female   
997  727-02-1313      A     Yangon        Member    Male   
998  347-56-2442      A     Yangon        Normal    Male   
999  849-09-3807      A     Yangon        Member  Female   

               Product line  Unit price  Quantity 

In [64]:
cols

['Invoice ID',
 'Branch',
 'City',
 'Customer type',
 'Gender',
 'Product line',
 'Unit price',
 'Quantity',
 'Tax 5%',
 'Total',
 'Date',
 'Time',
 'Payment',
 'cogs',
 'gross margin percentage',
 'gross income',
 'Rating']